
### Klasa GroundStation
Ta klasa reprezentuje stację naziemną na planecie. Stacje naziemne są kluczowe dla operacji satelitarnych, w tym komunikacji, przesyłu danych i śledzenia telemetrii.

- **Atrybuty**:
    - `name`: Łańcuch znaków reprezentujący nazwę stacji naziemnej.
    - `latitude`: Geograficzna szerokość stacji naziemnej w stopniach. Wartości dodatnie wskazują półkulę północną, a wartości ujemne półkulę południową.
    - `longitude`: Geograficzna długość stacji naziemnej w stopniach. Wartości dodatnie wskazują wschód od południka zerowego, a wartości ujemne zachód.


### Klasa Planet
Klasa `Planet` modeluje ciało niebieskie, dostarczając niezbędnych atrybutów do symulowania orbit i interakcji z satelitami.

- **Atrybuty**:
    - `name`: Nazwa planety.
    - `radius`: Średni promień planety w metrach. Ta wartość jest kluczowa do obliczania orbit i widoczności z powierzchni planety.
    - `mu`: Standardowy parametr grawitacyjny $(m^3/s^2)$, który jest produktem stałej grawitacyjnej $(G)$ i masy planety $(M)$. Jest kluczowy do obliczania siły grawitacyjnej wywieranej przez planetę.
    - `rotation_period`: Gwiazdowy dzień planety w sekundach, określający czas potrzebny planecie do wykonania jednego obrotu względem gwiazd.
    - `j2`: Bezmiarowy współczynnik reprezentujący spłaszczenie planety lub czynnik spłaszczenia z powodu jej rotacji. Ma to wpływ na orbity satelitów wokół planety.
    - `angular_velocity`: Kątowa prędkość obrotu planety w radianach na sekundę. Jest używana do obliczania ścieżki naziemnej satelitów.
    - `ground_stations`: Lista przechowująca instancje klasy `GroundStation`, reprezentujące stacje naziemne zlokalizowane na planecie.

- **Metody**:
    - `add_ground_station(name, latitude, longitude)`: Dodaje nowy obiekt `GroundStation` do listy `ground_stations`, rozszerzając sieć stacji naziemnych na planecie.
    - `is_satellite_visible(satellite)`: Iteruje po wszystkich stacjach naziemnych, aby ustalić, czy któraś z nich ma linię widzenia do danego satelity. Ta metoda zwykle wykorzystuje bieżącą pozycję satelity i lokalizację stacji naziemnej do obliczenia widoczności, biorąc pod uwagę takie czynniki, jak krzywizna planety i wysokość satelity.

# Jak określona jest klasa orbita i jak propagujemy orbitę?

- $\mu$: Standardowy parametr grawitacyjny planety (Standard gravitational parameter of the planet) $(m^3/s^2)$.
- $a$: Półoś wielka orbity (Semi-major axis of the orbit) $(m)$.
- $e$: Mimośród orbity (Eccentricity of the orbit).
- $\theta$: Prawdziwa anomalia (True anomaly) $(rad)$, kąt od kierunku periapsis do aktualnej pozycji satelity.
- $\Omega$: Prawoskrętność wznoszenia węzła wstępującego (Right Ascension of the Ascending Node, RAAN) $(rad)$.
- $i$: Inklinacja orbity (Inclination of the orbit) $(rad)$.
- $\omega$: Argument periapsis (Argument of periapsis) $(rad)$.

Proces obejmuje dwa główne kroki:

**Krok 1: Oblicz Pozycję i Prędkość w Współrzędnych Peryfokalnych**

1. Pół-latus rectum (Semi-latus rectum):
$$ p = a(1 - e^2) $$

2. Promień (Radius, odległość od punktu ogniskowego do satelity):
$$ r = \frac{p}{1 + e\cos(\theta)} $$

3. Wektor pozycji w ramie PQW (Position vector in PQW frame):
$$ \vec{r}_{pqw} = 
\begin{bmatrix}
r\cos(\theta) \\
r\sin(\theta) \\
0
\end{bmatrix}
$$

4. Wektor prędkości w ramie PQW (Velocity vector in PQW frame):
$$ \vec{v}_{pqw} = 
\begin{bmatrix}
-\sqrt{\frac{\mu}{p}}\sin(\theta) \\
\sqrt{\frac{\mu}{p}}(e + \cos(\theta)) \\
0
\end{bmatrix}
$$

**Krok 2: Obrót do Ramy ECI (Rotate to ECI Frame) Za Pomocą Trzech Macierzy Rotacji**

1. Obrót wokół osi Z o $-\Omega$ (RAAN, Rotation about the Z-axis by $-\Omega$):
$$ R_3(-\Omega) = 
\begin{bmatrix}
\cos(-\Omega) & -\sin(-\Omega) & 0 \\
\sin(-\Omega) & \cos(-\Omega) & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

2. Obrót wokół osi X o $-i$ (inclination, Rotation about the X-axis by $-i$):
$$ R_1(-i) = 
\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos(-i) & -\sin(-i) \\
0 & \sin(-i) & \cos(-i)
\end{bmatrix}
$$

3. Obrót wokół osi Z o $-\omega$ (argument of perigee, Rotation about the Z-axis by $-\omega$):
$$ R_3(-\omega) = 
\begin{bmatrix}
\cos(-\omega) & -\sin(-\omega) & 0 \\
\sin(-\omega) & \cos(-\omega) & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

**Ostateczna Pozycja i Prędkość w Ramie ECI (Final Position and Velocity in ECI Frame):**
$$ \vec{r}_{eci} = R_3(-\Omega)R_1(-i)R_3(-\omega)\vec{r}_{pqw} $$
$$ \vec{v}_{eci} = R_3(-\Omega)R_1(-i)R_3(-\omega)\vec{v}_{pqw} $$



# Dalej klasie Orbita

Funkcja `propagate_orbit` symuluje orbitę satelity w czasie, uwzględniając zarówno główną atrakcję grawitacyjną planety, jak i zakłócenie spowodowane spłaszczeniem planety (zakłócenie J2). Proces ten obejmuje rozwiązanie układu równań różniczkowych opisujących ruch satelity pod wpływem tych sił.

### Problem dwóch ciał z zakłóceniem J2

Problem dwóch ciał zwykle opisuje oddziaływanie grawitacyjne między dwoma ciałami, zakładając, że są one masami punktowymi, co prowadzi do orbit eliptycznych. Jednak rzeczywiste ciała niebieskie nie są masami punktowymi ani doskonałymi sferami, co prowadzi do dodatkowych efektów perturbacyjnych na orbicie satelity. Jedno z istotnych zakłóceń pochodzi od terminu J2, który uwzględnia wypukłość równikową planety spowodowaną jej rotacją.

#### Czym jest zakłócenie J2?

Zakłócenie J2 uwzględnia niesferyczny kształt planety. Potencjał grawitacyjny ciała z wypukłością równikową różni się od tego dla doskonałej sfery, co prowadzi do zmian w orbicie satelity w czasie, szczególnie wpływając na jej inklinację i prawoskrętność wznoszenia węzła wstępującego (RAAN).

### Sformułowanie matematyczne

Mając dane:
- $ \mu $: Standardowy parametr grawitacyjny planety $(m^3/s^2)$.
- $ J2 $: Bezmiarowy współczynnik reprezentujący wypukłość równikową planety.
- $ r $: Wektor pozycji satelity.
- $ v $: Wektor prędkości satelity.
- $ R $: Średni promień planety.

#### Równania różniczkowe dla propagacji orbity

Ruch satelity jest regulowany przez drugie prawo Newtona, z przyspieszeniem danym przez gradient potencjału grawitacyjnego. Uwzględniając zakłócenie J2, przyspieszenie $ a $ jest:
$$ \vec{a} = \vec{a}_{gravity} + \vec{a}_{J2} $$

Gdzie:
- $ \vec{a}_{gravity} = -\frac{\mu}{r^3} \vec{r} $ jest głównym przyspieszeniem grawitacyjnym.
- $ \vec{a}_{J2} $ uwzględnia zakłócenie spowodowane przez termin J2.

Wyraźnie, składowe przyspieszenia J2 w kartezjańskich współrzędnych są:
$$ \vec{a}_{J2} = \frac{1.5 J2 \mu R^2}{r^4} 
\left( \begin{array}{c}
(1 - 5\frac{z^2}{r^2})x \\
(1 - 5\frac{z^2}{r^2})y \\
(3 - 5\frac{z^2}{r^2})z
\end{array} \right)
$$

Ta formuła dostosowuje przyspieszenie satelity ze względu na wypukłość równikową, wpływając na ewolucję orbity.

#### Numeryczne rozwiązanie problemu

Aby propagować orbitę, numerycznie całkujemy równania ruchu za pomocą `solve_ivp`. Początkowy wektor stanu ($ \vec{y}_0 = [\vec{r}, \vec{v}] $) jest określany na podstawie aktualnych elementów orbitalnych satelity. Równania różniczkowe zdefiniowane przez $ \vec{a} $ są całkowane na żądanym przedziale czasu, dając trajektorię satelity.

### Obliczanie ścieżki naziemnej

Metoda `calculate_ground_track` przekształca trajektorię satelity z ramy ECI na współrzędne geograficzne (szerokość i długość geograficzna). Obliczenie to uwzględnia rotację Ziemi, pozwalając na określenie, gdzie satelita będzie umiejscowiony względem powierzchni planety w czasie.

Transformacja obejmuje:
1. Obliczanie długości geograficznej przez dostosowanie długości geograficznej ECI satelity przez rotację Ziemi w czasie.
2. Bezpośrednie obliczanie szerokości geograficznej z koordynatu z ECI satelity.
3. Dostosowanie długości geograficznej, aby upewnić się, że pozostaje ona w zakresie $[-180^\circ, 180^\circ]$.

Ta metoda dostarcza ścieżkę naziemną satelity, która jest kluczowa dla planowania misji, w tym okien obserwacyjnych i komunikacyjnych.

Podsumowując, `propagate_orbit` i `calculate_ground_track` razem oferują kompleksową symulację orbit satelitów, uwzględniając zarówno główne siły grawitacyjne, jak i zakłócenia spowodowane przez kształt planety, oraz mapując ścieżkę satelity nad powierzchnią planety.



Ostateczny kształt równania ruchu uwzględniający zakłócenie J2, które jest rozwiązywane przez numeryczny solver w funkcji `propagate_orbit`, łączy zarówno główną siłę grawitacyjną, jak i dodatkowe przyspieszenie spowodowane przez zakłócenie J2. Skutkuje to zestawem równań różniczkowych opisujących ruch satelity pod wpływem tych połączonych efektów.

Mając dane:
- $ \mu $: Standardowy parametr grawitacyjny planety $(m^3/s^2)$.
- $ R $: Średni promień planety (m).
- $ J2 $: Bezmiarowy współczynnik reprezentujący wypukłość równikową planety.
- $ \vec{r} = [x, y, z]^T $: Wektor pozycji satelity w ramie ECI (m).
- $ \vec{v} = [\dot{x}, \dot{y}, \dot{z}]^T $: Wektor prędkości satelity w ramie ECI $(m/s)$.

Wektor przyspieszenia $\vec{a}$ spowodowany grawitacją można wyrazić jako:
$$ \vec{a}_{gravity} = -\frac{\mu}{r^3} \vec{r} $$

Dla zakłócenia J2 składowe przyspieszenia są:
$$ \vec{a}_{J2} = \frac{1.5 J2 \mu R^2}{r^7} 
\left( \begin{array}{c}
(1 - 5\frac{z^2}{r^2})x \\
(1 - 5\frac{z^2}{r^2})y \\
(3 - 5\frac{z^2}{r^2})z
\end{array} \right)
$$

Całkowite przyspieszenie $\vec{a}$ jest sumą $\vec{a}_{gravity}$ i $\vec{a}_{J2}$, prowadząc do następujących równań różniczkowych drugiego rzędu dla ruchu satelity:

$$ \ddot{x} = -\frac{\mu x}{r^3} + \frac{1.5 J2 \mu R^2}{r^7} (1 - 5\frac{z^2}{r^2})x $$
$$ \ddot{y} = -\frac{\mu y}{r^3} + \frac{1.5 J2 \mu R^2}{r^7} (1 - 5\frac{z^2}{r^2})y $$
$$ \ddot{z} = -\frac{\mu z}{r^3} + \frac{1.5 J2 \mu R^2}{r^7} (3 - 5\frac{z^2}{r^2})z $$

Gdzie $ r = \sqrt{x^2 + y^2 + z^2} $ jest wielkością wektora pozycji $\vec{r}$.

Te równania są wyrażone w terminach składowych pozycji i prędkości satelity i muszą być scałkowane w czasie, aby symulować orbitę. Do tego celu używany jest numeryczny solver `solve_ivp` z SciPy, który wymaga sformułowania równań jako system pierwszego rzędu. Jest to zwykle realizowane przez zdefiniowanie wektora stanu $\vec{y} = [\vec{r}, \vec{v}]^T$ i wyrażenie równań ruchu w formie:

$$ \frac{d\vec{y}}{dt} = \left[ \begin{array}{c}
\vec{v} \\
\vec{a}
\end{array} \right] $$

Gdzie $\vec{a}$ jest całkowitym przyspieszeniem, włączając zarówno $\vec{a}_{gravity}$ jak i $\vec{a}_{J2}$. Ten system pierwszego rzędu jest tym, co `solve_ivp` rozwiązuje, aby propagować orbitę.

